# Lightcurve

We will now check if the emission is variable over time. We will make use of the spectrum datasets that we have created ealier.

## Imports
We will start with the import of a few modules.

In [ ]:
import matplotlib.pyplot as plt

import numpy as np

from numpy import sqrt

import astropy.units as u

from gammapy.datasets import Datasets

from gammapy.modeling.models import Models

In [ ]:
from gammapy.modeling import Fit

fit = Fit()

## Load Datasets

We will now use the run-by-run data sets that we have created earlier.

In [ ]:
spectrum_datasets = Datasets.read('SpectrumDatasets.yaml')

In [ ]:
spectrum_datasets

We will also use our best-fit spectral model.

In [ ]:
bestfit_spectral = Models.read('SpectrumBestFit.fits.gz')

In [ ]:
bestfit_spectral

We will set this model as our current model for the dataset.

In [ ]:
spectrum_datasets.models = bestfit_spectral

In [ ]:
spectrum_datasets.models

And we can re-run the fit. This time the same model will be fit simultanously to all individual datasets. This is called a joint fit.

In [ ]:
fit.run(spectrum_datasets)

In [ ]:
spectrum_datasets.models.parameters.to_table().show_in_notebook()

## Lightcurve Estimator

Gammapy provides a very powerful tool to obtain light curves. The LightcurveEstimator.

In [ ]:
from gammapy.estimators import LightCurveEstimator

We will need to tell the LightCurveEstimator the name of the model that is to be analysed.

In [ ]:
spectrum_datasets.models[0].name

In [ ]:
lc_maker = LightCurveEstimator(source = spectrum_datasets.models[0].name)

In [ ]:
lc = lc_maker.run(spectrum_datasets)

In [ ]:
lc

In [ ]:
lc.to_table()

In [ ]:
flux = bestfit_spectral[0].spectral_model.integral(0.8803178367920126*u.TeV,
                                                   10.573712634405641*u.TeV)

In [ ]:
flux

In [ ]:
ax = lc.plot(sed_type = 'flux')

ax.axhline(flux)

We can also calculate the sum of the chi2 values to calculate the probabilty:

In [ ]:
lc_table = lc.to_table(sed_type = 'flux')

In [ ]:
lc_table.colnames

In [ ]:
chi2 = np.sum(((lc_table['flux'] - flux) / lc_table['flux_err'])**2)

In [ ]:
import scipy.stats

In [ ]:
P = scipy.stats.chi2.sf(chi2, len(spectrum_datasets) - 1)

print('probabilty: ',P)

In [ ]:
print('significant?', P < 2.7e-3)

## Manual Binning of the Light Curve

Let's try to bin observations done in the same night into on data point.

In [ ]:
spectrum_datasets.gti.time_start

In [ ]:
from astropy.time import Time

In [ ]:
time_intervals = []

t_start = np.min(spectrum_datasets.gti.time_start)
t_stop = np.max(spectrum_datasets.gti.time_stop)

In [ ]:
t_start

In [ ]:
# Let's start at noon

t = int(t_start.value)+0.5

In [ ]:
while t < t_stop.value :
    t2 = t + 1
    time_intervals.append(Time([t, t2], format=t_start.format, scale=t_start.scale))
    t = t2

In [ ]:
time_intervals

In [ ]:
lc_maker = LightCurveEstimator(source = spectrum_datasets.models[0].name,
                               time_intervals = time_intervals
                              )

In [ ]:
lc = lc_maker.run(spectrum_datasets)

In [ ]:
lc.to_table()

In [ ]:
ax = lc.plot(sed_type = 'flux')

ax.axhline(flux)

## Summary

We have seen how we can easily obtain a light curve of the emission and check whether it is variable or not. Gamma-ray emission can be highly variable (AGNe for example) or periodic (binary systems for example). Gamma-ray sources which are resolved, i.e. they are seen as extended, are not variable at the short time scales probed here.

We have checked for variability on a run-by-run basis. The rebinning we have seen allows only the combination of runs into larger bins. Testing for variability on shorter time scales (less than a run) will need a different approach. Testing for periodicity of pular emission (seconds to milliseconds) will require corrections for the movement of Earth around the Sun.